In [11]:
from transformers import AutoModelForSeq2SeqLM
import peft
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, IA3Config, TaskType
import torch
from datasets import load_dataset
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
from tqdm import tqdm
from datasets import load_dataset

device = "cuda"
model_name_or_path = "bigscience/mt0-large"
tokenizer_name_or_path = "bigscience/mt0-large"

checkpoint_name = "financial_sentiment_analysis_ia3_v1.pt"
text_column = "sentence"
label_column = "text_label"
max_length = 128
lr = 8e-3
num_epochs = 3
batch_size = 8

In [12]:
import importlib

importlib.reload(peft)

<module 'peft' from '/nfs/hpc/share/balasuri/peft/src/peft/__init__.py'>

In [15]:
# creating model
peft_config = IA3Config(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, feedforward_modules=[])

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

Some weights of UMT5ForConditionalGeneration were not initialized from the model checkpoint at bigscience/mt0-large and are newly initialized: ['encoder.block.9.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.8.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.3.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.20.layer.0.SelfAttention.relative_attention_bias.weight', 'encoder.block.13.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.11.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.10.layer.0.SelfAttention.relative_attention_bias.weight', 'encoder.block.8.layer.0.SelfAttention.relative_attention_bias.weight', 'encoder.block.22.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.19.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.22.layer.0.SelfAttention.relative_attention_bias.weight', 'encoder.block.17.layer.0.SelfAttention.relative_attentio

1229604864

In [14]:
model
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

1229604864


In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model

In [ ]:
# loading dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree")
dataset = dataset["train"].train_test_split(test_size=0.1)
dataset["validation"] = dataset["test"]
del dataset["test"]

classes = dataset["train"].features["label"].names
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["label"]]},
    batched=True,
    num_proc=1,
)

dataset["train"][0]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

{'sentence': 'There have not been previous share subscriptions with 2004 stock options .',
 'label': 1,
 'text_label': 'neutral'}

In [16]:
# data preprocessing
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=3, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs


processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
print(len(train_dataloader))
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

Running tokenizer on dataset:   0%|          | 0/2037 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/227 [00:00<?, ? examples/s]

255


In [8]:
# optimizer and lr scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [9]:
# training and evaluation
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:06<00:00,  4.48it/s]


epoch=0: train_ppl=tensor(1.7246, device='cuda:0') train_epoch_loss=tensor(0.5450, device='cuda:0') eval_ppl=tensor(1.1729, device='cuda:0') eval_epoch_loss=tensor(0.1595, device='cuda:0')


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:03<00:00,  9.00it/s]


epoch=1: train_ppl=tensor(1.2072, device='cuda:0') train_epoch_loss=tensor(0.1883, device='cuda:0') eval_ppl=tensor(1.1567, device='cuda:0') eval_epoch_loss=tensor(0.1455, device='cuda:0')


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:03<00:00,  9.02it/s]

epoch=2: train_ppl=tensor(1.1585, device='cuda:0') train_epoch_loss=tensor(0.1471, device='cuda:0') eval_ppl=tensor(1.1579, device='cuda:0') eval_epoch_loss=tensor(0.1466, device='cuda:0')


In [10]:
# print accuracy
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["validation"]["text_label"]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{dataset['validation']['text_label'][:10]=}")

accuracy=89.42731277533039 % on the evaluation dataset
eval_preds[:10]=['positive', 'neutral', 'negative', 'neutral', 'positive', 'neutral', 'positive', 'negative', 'negative', 'positive']
dataset['validation']['text_label'][:10]=['neutral', 'neutral', 'negative', 'neutral', 'positive', 'neutral', 'positive', 'negative', 'negative', 'neutral']


In [ ]:
# saving model
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

In [ ]:
ckpt = f"{peft_model_id}/adapter_model.bin"
!du -h $ckpt

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
model.eval()
i = 13
inputs = tokenizer(dataset["validation"][text_column][i], return_tensors="pt")
print(dataset["validation"][text_column][i])
print(inputs)

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))